In [43]:
import pandas as pd
from sklearn import model_selection, metrics, preprocessing
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np

In [44]:
header = ['user_id', 'item_id', 'rating', 'timestamp']
df = pd.read_csv('ml-100k/u.data', sep='\t', names=header)
df = df.drop('timestamp', axis=1)

In [45]:
df_user = df.loc[df["user_id"] == 196]
df_user[:5]

user_id  item_id  rating
0         196      242       3
940       196      393       4
1133      196      381       4
1812      196      251       3
1896      196      655       5

In [46]:
class DataSet(torch.utils.data.Dataset):
    def __init__(self, users, items, ratings):
        super(DataSet, self).__init__()
        self.users = users
        self.items = items
        self.ratings = ratings
        
    def __getitem__(self, idx):
        user = self.users[idx]
        item = self.items[idx]
        rating = self.ratings[idx]
        
        return (
            torch.tensor(user, dtype=torch.long),
            torch.tensor(item, dtype=torch.long),
            torch.tensor(rating, dtype=torch.long),
        )

In [47]:
train, test = model_selection.train_test_split(df, test_size=0.1, random_state=4, stratify=df.rating.values)

train.head()

user_id  item_id  rating
53270      682      423       5
76926      668      137       3
86402      643      176       5
42887      561      216       3
72708      854        8       5

In [48]:
train_data = DataSet(train.user_id.values, train.item_id.values, train.rating.values)

In [49]:
class net(nn.Module):
    def __init__(self, num_users, num_movies):
        super(net, self).__init__()
        
        self.user_embed = nn.Embedding(num_users, 32)
        self.movie_embed = nn.Embedding(num_movies, 32)
        self.out = nn.Linear(64, 1)
        
    def forward(self, users, movies):
        user_embeds = self.user_embed(users)
        movie_embeds = self.movie_embed(movies)
        
        #print(user_embeds, user_embeds.shape)
        #print(movie_embeds, movie_embeds.shape)

        output = torch.cat([torch.reshape(user_embeds, (1, len(user_embeds))), torch.reshape(movie_embeds, (1, len(movie_embeds)))], dim=1)
        
        output = self.out(output)

        return output

In [50]:
num_unique_users = len(set(df.user_id.values))
num_unique_movies = len(set(df.item_id.values))

In [51]:
num_unique_users

943

In [52]:
num_unique_movies

1682

In [53]:
loss_function = nn.MSELoss(reduction='none')

In [54]:
model = net(num_unique_users, num_unique_movies)

In [55]:
opt = torch.optim.Adam(model.parameters(), lr=1e-3)

In [56]:
print(model)

net(
  (user_embed): Embedding(943, 32)
  (movie_embed): Embedding(1682, 32)
  (out): Linear(in_features=64, out_features=1, bias=True)
)


In [57]:
#loss_function = nn.BCELoss()

In [58]:
train

user_id  item_id  rating
53270      682      423       5
76926      668      137       3
86402      643      176       5
42887      561      216       3
72708      854        8       5
...        ...      ...     ...
24793       70      483       5
7227       318      396       1
79082      742      546       1
31769      393      173       5
23153      358      863       5

[90000 rows x 3 columns]

In [59]:
model.train()
total_loss = 0



for user, item, label in train_data:

    print(user, item, label)
    
    opt.zero_grad()
    output = model(user, item)
    
    #print(output)
    #print(label)
    
    print(output.shape, torch.tensor([label]).shape)
    
    
    label = torch.tensor([label])
    loss = F.nll_loss(output, label)
    loss.backward()
    opt.step()

    #print(loss)
    #total_loss += loss.item()

#losses.append(total_loss)

tensor(682) tensor(423) tensor(5)
torch.Size([1, 1]) torch.Size([1])


IndexError: Target 5 is out of bounds.